In [1]:
!pip install pyspark
import os
import pyspark
from pyspark import SparkFiles


# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-17-jdk-headless -qq > /dev/null
!wget -q http://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2"

# Start a SparkSession
import findspark
findspark.init()

!wget https://jdbc.postgresql.org/download/postgresql-42.4.1.jar

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.4.1.jar").getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:5 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu

In [2]:
# Read in data from S3 Buckets

url="https://jewelry-reviews.s3.us-west-1.amazonaws.com/amazon_reviews_us_Jewelry_v1_00.tsv"
spark.sparkContext.addFile(url)
amazon_df = spark.read.csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

# Show DataFrame
amazon_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...|2015-08-31 00:00:00|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...| 

In [3]:
from pyspark.sql.functions import to_date
amazon_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   50423057|R135Q3VZ4DQN5N|B00JWXFDMG|     657335467|Everbling Purple ...|         Jewelry|          5|            0|          0|   N|                Y|           Beauties!|so beautiful even...|2015-08-31 00:00:00|
|         US|   11262325|R2N0QQ6R4T7YRY|B00W5T1H9W|      26030170|925 Sterling Silv...| 

In [4]:
# Create the customers_table DataFrame
customers_df = amazon_df.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id", "customer_count")
customers_df.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   45657423|                 1|
|   47663761|                 1|
|   11931196|                 1|
|   44620216|                 1|
|    1167081|                 1|
|   17682867|                 1|
|   10782454|                 1|
|   51881951|                 1|
|    1537684|                 1|
|   17133428|                 5|
|     661760|                 1|
|    9867391|                 9|
|   31268594|                 1|
|   48581590|                 1|
|   32466409|                 3|
|   38490515|                 1|
|   41318181|                 4|
|   18800155|                 6|
|   46306239|                 1|
|   28409312|                 1|
+-----------+------------------+
only showing top 20 rows



In [5]:
# Create the products_table DataFrame and drop duplicates. 
products_df = amazon_df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00LHJMHVS|FAIRY COUPLE Wome...|
|B00TPWQ6I4|Chic Alert Medica...|
|B00P837QR4|The Original Roll...|
|B00L280CN4|10pcs 18" 18 Inch...|
|B00KGU5T4Q|KONOV Mens Stainl...|
|B00JHF388K|Sterling Silver w...|
|B00FB05CES|CHOOSE YOUR COLOR...|
|B00D4KUMQ0|INBLUE Men's Stai...|
|B00GTCD2X0|Beautiful Carved ...|
|B00QUHZQP6|Authentic Chamili...|
|B00NJ317T8|Smitco LLC Silver...|
|B00658M8EC|925 Sterling Silv...|
|B00L43PV2Y|Hydia Steel Jewel...|
|B00U24CSP4|Platinum-Plated S...|
|B006OHTHZ2|316L Stainless St...|
|B00QW72KDA|PiercingJ Pair 12...|
|B006VN35L6|5mm Stainless Ste...|
|B00CGID7PY|925 Sterling Silv...|
|B00TXHSXWI|Silver Forest Kno...|
|B004Z1EQRS|PammyJ Silvertone...|
+----------+--------------------+
only showing top 20 rows



In [6]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = amazon_df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show(10)

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R135Q3VZ4DQN5N|   50423057|B00JWXFDMG|     657335467| 2015-08-31|
|R2N0QQ6R4T7YRY|   11262325|B00W5T1H9W|      26030170| 2015-08-31|
|R3N5JE5Y4T6W5M|   27541121|B00M2L6KFY|     697845240| 2015-08-31|
|R2I150CX5IVY9Q|    5350721|B0006SW2WU|     569859289| 2015-08-31|
|R1RM9ICOOA9MQ3|   24484424|B009YPDW70|     332947422| 2015-08-31|
|R2J2KMDL10UMSH|     457767|B00QGA8XZM|     424804582| 2015-08-31|
|R3R9ZUFA4TB4FQ|   31009358|B00L0N0E8E|     589441783| 2015-08-31|
|R3UQ8VAQN7R6WL|    7034628|B00DMWQKWA|     517428445| 2015-08-31|
|R1FXZ69C01JNQM|   34608636|B00THLHAGA|     320313750| 2015-08-31|
| RY36LB4OW0FFS|   44972687|B00UXSEOHY|     589365488| 2015-08-31|
+--------------+-----------+----------+--------------+-----------+
only showing top 10 rows



In [7]:
# Create the vine_table. DataFrame
vine_df = amazon_df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R135Q3VZ4DQN5N|          5|            0|          0|   N|                Y|
|R2N0QQ6R4T7YRY|          5|            0|          0|   N|                N|
|R3N5JE5Y4T6W5M|          5|            0|          0|   N|                Y|
|R2I150CX5IVY9Q|          5|            0|          0|   N|                Y|
|R1RM9ICOOA9MQ3|          5|            0|          0|   N|                Y|
|R2J2KMDL10UMSH|          5|            0|          0|   N|                Y|
|R3R9ZUFA4TB4FQ|          5|            0|          0|   N|                Y|
|R3UQ8VAQN7R6WL|          5|            0|          0|   N|                Y|
|R1FXZ69C01JNQM|          5|            0|          0|   N|                Y|
| RY36LB4OW0FFS|          5|            0|          0|   N|     

In [10]:
# Store environmental variable
from getpass import getpass
password = getpass('ENTER DATABASE PASSWORD ')

# Configure settings for RDS
mode = "append"
jdbc_url = "jdbc:postgresql://dataviz.c6ro7zigdq6s.us-west-1.rds.amazonaws.com:5432/postgres"
config = {"user":"jeffbaizo", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

ENTER DATABASE PASSWORD ··········


In [11]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [12]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [13]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [14]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)